In [ ]:
pip install flexvalue --upgrade

### Change the year below from 2020 to 2021 to see the results using the 2021 ACC.

In [ ]:
year = '2020'
!flexvalue download-avoided-costs-data-db --year $year

In [ ]:
from google.colab import files
import pandas as pd
from io import BytesIO
pd.set_option('display.max_rows', None)

In [ ]:
print("You must upload a user inputs file. To see an example, try out:")
print("https://storage.googleapis.com/flexvalue-public-resources/examples/example_user_inputs_metered.csv")

In [ ]:
user_inputs_dict = files.upload()

if len(user_inputs_dict)!=1:
    print("Must select exactly one file")
else:
    user_inputs = pd.read_csv(BytesIO(list(user_inputs_dict.values())[0]))
    user_input_filename = list(user_inputs_dict.keys())[0]
metered_load_shape_df = None

In [ ]:
user_inputs

In [ ]:
print("You can optionally upload a custom metered load shape. If using the previous example, also upload this file:")
print("https://storage.googleapis.com/flexvalue-public-resources/examples/example_metered_load_shapes.csv")
print("If you want to use a custom load shape, either don't run the following cell, or run it and hit 'Cancel Upload'")

In [ ]:
metered_ls_dict = files.upload()
if len(metered_ls_dict)==0:
    metered_load_shape_df = None
elif len(metered_ls_dict) == 1:
    metered_load_shape = list(metered_ls_dict.values())[0]
    metered_load_shape_df = pd.read_csv(BytesIO(metered_load_shape)).set_index('hour_of_year')
else:
    print("Must select at most one file")

In [ ]:
from flexvalue import FlexValueRun

flx_run = FlexValueRun(database_year=year, metered_load_shape=metered_load_shape_df)

outputs_table, outputs_table_totals, electric_benefits, gas_benefits = flx_run.get_results(user_inputs=user_inputs)
outputs_table

In [ ]:
outputs_table_totals

In [ ]:
# Run this cell if you want to download the FLEXvalue output.
from pathlib import Path
user_input_filename_no_ext = Path(user_input_filename).stem
outputs_filename = f'{user_input_filename_no_ext}_outputs.csv'
outputs_table.to_csv(outputs_filename)
files.download(outputs_filename)

In [ ]:
from flexvalue.plots import plot_results
plot_results(outputs_table_totals, electric_benefits, gas_benefits)